In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pickle
import seaborn as sns
import time
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from numpy import concatenate
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.semi_supervised import LabelPropagation
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import pyspark
import random
import pandas as pd
import sqlite3
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder

In [3]:
df = pd.read_csv('java_Large.csv')
c = sqlite3.connect('jupyter_sql_tutorial.db')

In [ ]:
#df.to_sql('Q1', c)

In [4]:
%load_ext sql

%sql sqlite:///jupyter_sql_tutorial.db

In [27]:
%%sql
DROP TABLE Q1


 * sqlite:///jupyter_sql_tutorial.db
Done.


[]

In [28]:
%%sql
DROP TABLE Q2

 * sqlite:///jupyter_sql_tutorial.db
Done.


[]

In [7]:
df.head()

,Id,QueryString,TopClickedUrls,PopularityRank,PredictedLabel
0,ac94e1d0-6ca1-4967-80ec-6766f066c7b4,64 bit java,https://www.java.com/en/download/windows-64bit...,1,False
1,7b10d3b9-685a-419f-b2ff-0e337ed89f7d,java 64 bit download,https://www.java.com/en/download/windows-64bit...,2,False
2,55adb9c0-3afa-4404-a907-8061ece86cdd,java test,"https://www.java.com/verify/,https://www.java....",3,False
3,2ae036ca-9f56-43ed-a3a1-14894dcee7d2,java control panel windows 10,https://www.java.com/en/download/help/win_cont...,4,False
4,8b03c343-3279-4760-ac07-6f0bbbf34a58,java jre download,"https://java.com/en/download/,http://www.java....",5,False


In [8]:
def cleanText(text):
    
    lemma = WordNetLemmatizer()
    stp = stopwords.words('english')
    
    text = re.sub("[^a-zA-Z0-9]"," ",text)
    text = text.lower()

    text = nltk.word_tokenize(text)
    text = [lemma.lemmatize(word) for word in text]
    
    text = [word for word in text if word not in stp]
    
    text = " ".join(text)
    
    return text

In [9]:
cleanedText = []
for text in df["QueryString"]:   
    cleanedText.append(cleanText(text))
print(cleanedText[0])

64 bit java


In [31]:
y = df["PredictedLabel"]
y.shape

(118728,)

In [32]:
dt={'id': df["Id"], 'query': cleanedText, 'rank': df["PopularityRank"], 'urls': df["TopClickedUrls"], 'label': y}
df1 = pd.DataFrame(dt)
df1.head()

,id,query,rank,urls,label
0,ac94e1d0-6ca1-4967-80ec-6766f066c7b4,64 bit java,1,https://www.java.com/en/download/windows-64bit...,False
1,7b10d3b9-685a-419f-b2ff-0e337ed89f7d,java 64 bit download,2,https://www.java.com/en/download/windows-64bit...,False
2,55adb9c0-3afa-4404-a907-8061ece86cdd,java test,3,"https://www.java.com/verify/,https://www.java....",False
3,2ae036ca-9f56-43ed-a3a1-14894dcee7d2,java control panel window 10,4,https://www.java.com/en/download/help/win_cont...,False
4,8b03c343-3279-4760-ac07-6f0bbbf34a58,java jre download,5,"https://java.com/en/download/,http://www.java....",False


In [33]:
df1["len"] = [len(text) for text in df1["query"].values]

In [34]:
df1.head()

,id,query,rank,urls,label,len
0,ac94e1d0-6ca1-4967-80ec-6766f066c7b4,64 bit java,1,https://www.java.com/en/download/windows-64bit...,False,11
1,7b10d3b9-685a-419f-b2ff-0e337ed89f7d,java 64 bit download,2,https://www.java.com/en/download/windows-64bit...,False,20
2,55adb9c0-3afa-4404-a907-8061ece86cdd,java test,3,"https://www.java.com/verify/,https://www.java....",False,9
3,2ae036ca-9f56-43ed-a3a1-14894dcee7d2,java control panel window 10,4,https://www.java.com/en/download/help/win_cont...,False,28
4,8b03c343-3279-4760-ac07-6f0bbbf34a58,java jre download,5,"https://java.com/en/download/,http://www.java....",False,17


In [35]:
df1.to_sql('Q1', c)

In [38]:
%%time
%%sql
SELECT *
FROM Q1
WHERE len = 20 AND label = False AND query ='download 64 bit java'

 * sqlite:///jupyter_sql_tutorial.db
Done.
Wall time: 78 ms


index,id,query,rank,urls,label,len
356,e9592dfd-e7fe-4037-a774-3bd2a4d28be9,download 64 bit java,284,"https://www.java.com/en/download/windows-64bit.jsp,https://www.java.com/en/download/faq/java_win64bit.xml,https://download.cnet.com/Java-Runtime-Environment-JRE-64-Bit/3000-2213_4-75317067.html",0,20
702,40d8e511-0f39-4fa5-8ec1-0a3a8af781fc,download 64 bit java,422,"https://www.java.com/en/download/faq/java_win64bit.xml,https://www.java.com/en/download/windows-64bit.jsp,https://tradownload.ws/results/java-7-64-bit-install.html",0,20
1305,0555cb13-1e2b-47e2-8133-ae167f85ee4f,download 64 bit java,511,"https://www.java.com/en/download/windows-64bit.jsp,https://www.java.com/en/download/faq/java_win64bit.xml,http://www.java.com/getjava/",0,20
6952,e9592dfd-e7fe-4037-a774-3bd2a4d28be9,download 64 bit java,284,"https://www.java.com/en/download/windows-64bit.jsp,https://www.java.com/en/download/faq/java_win64bit.xml,https://download.cnet.com/Java-Runtime-Environment-JRE-64-Bit/3000-2213_4-75317067.html",0,20
7298,40d8e511-0f39-4fa5-8ec1-0a3a8af781fc,download 64 bit java,422,"https://www.java.com/en/download/faq/java_win64bit.xml,https://www.java.com/en/download/windows-64bit.jsp,https://tradownload.ws/results/java-7-64-bit-install.html",0,20
7901,0555cb13-1e2b-47e2-8133-ae167f85ee4f,download 64 bit java,511,"https://www.java.com/en/download/windows-64bit.jsp,https://www.java.com/en/download/faq/java_win64bit.xml,http://www.java.com/getjava/",0,20
13548,e9592dfd-e7fe-4037-a774-3bd2a4d28be9,download 64 bit java,284,"https://www.java.com/en/download/windows-64bit.jsp,https://www.java.com/en/download/faq/java_win64bit.xml,https://download.cnet.com/Java-Runtime-Environment-JRE-64-Bit/3000-2213_4-75317067.html",0,20
13894,40d8e511-0f39-4fa5-8ec1-0a3a8af781fc,download 64 bit java,422,"https://www.java.com/en/download/faq/java_win64bit.xml,https://www.java.com/en/download/windows-64bit.jsp,https://tradownload.ws/results/java-7-64-bit-install.html",0,20
14497,0555cb13-1e2b-47e2-8133-ae167f85ee4f,download 64 bit java,511,"https://www.java.com/en/download/windows-64bit.jsp,https://www.java.com/en/download/faq/java_win64bit.xml,http://www.java.com/getjava/",0,20
20144,e9592dfd-e7fe-4037-a774-3bd2a4d28be9,download 64 bit java,284,"https://www.java.com/en/download/windows-64bit.jsp,https://www.java.com/en/download/faq/java_win64bit.xml,https://download.cnet.com/Java-Runtime-Environment-JRE-64-Bit/3000-2213_4-75317067.html",0,20


In [39]:
indexed_df= df1.set_index(['label', 'len'])
sorted_ix_df=indexed_df.sort_index()
sorted_ix_df

id  \
label len                                         
False 4    d3408d25-3e05-46a1-8813-eb7760490ddb   
      4    c4c464f0-2c39-44a6-943f-d9902c7980e9   
      4    8d218964-c3d2-433f-9bfd-2975b6fa3775   
      4    97333ae7-0b10-45e9-8e09-17f30d6a2558   
      4    248b8d8a-c2b5-4249-8e5e-41e33414fc62   
...                                         ...   
True  135  15dd4725-a40b-491b-9f1f-7048eb40075d   
      135  15dd4725-a40b-491b-9f1f-7048eb40075d   
      135  15dd4725-a40b-491b-9f1f-7048eb40075d   
      135  15dd4725-a40b-491b-9f1f-7048eb40075d   
      135  15dd4725-a40b-491b-9f1f-7048eb40075d   

                                                       query  rank  \
label len                                                            
False 4                                                 java    30   
      4                                                 java    76   
      4                                                 java   155   
      4                                                 java   196   
      4                                                 java   201   
...                                                      ...   ...   
True  135  error 500 org springframework web util nesteds...   565   
      135  error 500 org springframework web util nesteds...   565   
      135  error 500 org springframework web util nesteds...   565   
      135  error 500 org springframework web util nesteds...   565   
      135  error 500 org springframework web util nesteds...   565   

                                                        urls  
label len                                                     
False 4    https://www.java.com/en/download/installed.jsp...  
      4    https://www.java.com/,http://oracle.com/java/,...  
      4    https://www.java.com/,https://www.java.com/inc...  
      4    https://www.java.com/,https://www.java.com/inc...  
      4    https://www.roseindia.net/help/java/o/java-ope...  
...                                                      ...  
True  135  https://stackoverflow.com/questions/34145128/h...  
      135  https://stackoverflow.com/questions/34145128/h...  
      135  https://stackoverflow.com/questions/34145128/h...  
      135  https://stackoverflow.com/questions/34145128/h...  
      135  https://stackoverflow.com/questions/34145128/h...  

[118728 rows x 4 columns]

In [40]:
sorted_ix_df.to_sql('Q2', c)

In [41]:
%reload_ext sql

%sql sqlite:///jupyter_sql_tutorial.db

In [57]:
%%time
%%sql
SELECT *
FROM Q2
WHERE len = 20 AND label = False AND query ='download 64 bit java'

 * sqlite:///jupyter_sql_tutorial.db
Done.
Wall time: 5 ms


label,len,id,query,rank,urls
0,20,e9592dfd-e7fe-4037-a774-3bd2a4d28be9,download 64 bit java,284,"https://www.java.com/en/download/windows-64bit.jsp,https://www.java.com/en/download/faq/java_win64bit.xml,https://download.cnet.com/Java-Runtime-Environment-JRE-64-Bit/3000-2213_4-75317067.html"
0,20,40d8e511-0f39-4fa5-8ec1-0a3a8af781fc,download 64 bit java,422,"https://www.java.com/en/download/faq/java_win64bit.xml,https://www.java.com/en/download/windows-64bit.jsp,https://tradownload.ws/results/java-7-64-bit-install.html"
0,20,0555cb13-1e2b-47e2-8133-ae167f85ee4f,download 64 bit java,511,"https://www.java.com/en/download/windows-64bit.jsp,https://www.java.com/en/download/faq/java_win64bit.xml,http://www.java.com/getjava/"
0,20,e9592dfd-e7fe-4037-a774-3bd2a4d28be9,download 64 bit java,284,"https://www.java.com/en/download/windows-64bit.jsp,https://www.java.com/en/download/faq/java_win64bit.xml,https://download.cnet.com/Java-Runtime-Environment-JRE-64-Bit/3000-2213_4-75317067.html"
0,20,40d8e511-0f39-4fa5-8ec1-0a3a8af781fc,download 64 bit java,422,"https://www.java.com/en/download/faq/java_win64bit.xml,https://www.java.com/en/download/windows-64bit.jsp,https://tradownload.ws/results/java-7-64-bit-install.html"
0,20,0555cb13-1e2b-47e2-8133-ae167f85ee4f,download 64 bit java,511,"https://www.java.com/en/download/windows-64bit.jsp,https://www.java.com/en/download/faq/java_win64bit.xml,http://www.java.com/getjava/"
0,20,e9592dfd-e7fe-4037-a774-3bd2a4d28be9,download 64 bit java,284,"https://www.java.com/en/download/windows-64bit.jsp,https://www.java.com/en/download/faq/java_win64bit.xml,https://download.cnet.com/Java-Runtime-Environment-JRE-64-Bit/3000-2213_4-75317067.html"
0,20,40d8e511-0f39-4fa5-8ec1-0a3a8af781fc,download 64 bit java,422,"https://www.java.com/en/download/faq/java_win64bit.xml,https://www.java.com/en/download/windows-64bit.jsp,https://tradownload.ws/results/java-7-64-bit-install.html"
0,20,0555cb13-1e2b-47e2-8133-ae167f85ee4f,download 64 bit java,511,"https://www.java.com/en/download/windows-64bit.jsp,https://www.java.com/en/download/faq/java_win64bit.xml,http://www.java.com/getjava/"
0,20,e9592dfd-e7fe-4037-a774-3bd2a4d28be9,download 64 bit java,284,"https://www.java.com/en/download/windows-64bit.jsp,https://www.java.com/en/download/faq/java_win64bit.xml,https://download.cnet.com/Java-Runtime-Environment-JRE-64-Bit/3000-2213_4-75317067.html"
